In [ ]:
import transformers

In [2]:
transformers.__version__

'4.30.2'

## load model directly

In [3]:
from transformers import AutoModel, AutoTokenizer

In [4]:
model_dir = '/root/.cache/huggingface/hub/THUDM/chatglm2-6b-32k'
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

> **8位量化版本**

In [5]:
model = AutoModel.from_pretrained(model_dir, trust_remote_code=True).quantize(8).half().cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
model = model.eval()

In [7]:
response, history = model.chat(tokenizer, "你好", history=[])

In [8]:
print(response)

你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。


In [9]:
history

[('你好', '你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。')]

In [10]:
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)

In [11]:
print(response)

晚上睡不着觉可能有很多原因，例如压力、焦虑、抑郁、不健康的睡眠习惯、药物副作用等等。以下是一些可能有帮助的建议:

1. 创建一个舒适的睡眠环境。保持卧室安静、凉爽、黑暗，并配备一个舒适的床和枕头。

2. 避免在睡前使用电子产品。手机、平板电脑和电视等蓝光屏幕会刺激大脑，使人难以入睡。

3. 建立一个固定的睡眠时间。尽量在每天的相同时间入睡和起床，帮助身体建立一个稳定的睡眠节律。

4. 避免在睡前饮用咖啡、茶、可乐或其他含咖啡因的饮料。这些饮料会刺激大脑，使人难以入睡。

5. 尝试放松技巧。例如深呼吸、渐进性肌肉松弛、瑜伽等，这些都可以帮助你放松身体和思维。

6. 避免在睡前进行剧烈的体力活动。例如剧烈运动或紧张的工作，这些都可能导致身体和大脑疲劳，使人难以入睡。

7. 避免使用太多的卧铺。如果躺在床上超过20分钟还不能入睡，那么最好起床，走到别处，做一些平静的活动，如阅读或听轻柔的音乐，直到感到困倦为止。

如果以上方法不能解决您的问题，最好咨询医生或专业的睡眠医学专家，获取更专业的建议和帮助。


In [12]:
history

[('你好', '你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。'),
 ('晚上睡不着应该怎么办',
  '晚上睡不着觉可能有很多原因，例如压力、焦虑、抑郁、不健康的睡眠习惯、药物副作用等等。以下是一些可能有帮助的建议:\n\n1. 创建一个舒适的睡眠环境。保持卧室安静、凉爽、黑暗，并配备一个舒适的床和枕头。\n\n2. 避免在睡前使用电子产品。手机、平板电脑和电视等蓝光屏幕会刺激大脑，使人难以入睡。\n\n3. 建立一个固定的睡眠时间。尽量在每天的相同时间入睡和起床，帮助身体建立一个稳定的睡眠节律。\n\n4. 避免在睡前饮用咖啡、茶、可乐或其他含咖啡因的饮料。这些饮料会刺激大脑，使人难以入睡。\n\n5. 尝试放松技巧。例如深呼吸、渐进性肌肉松弛、瑜伽等，这些都可以帮助你放松身体和思维。\n\n6. 避免在睡前进行剧烈的体力活动。例如剧烈运动或紧张的工作，这些都可能导致身体和大脑疲劳，使人难以入睡。\n\n7. 避免使用太多的卧铺。如果躺在床上超过20分钟还不能入睡，那么最好起床，走到别处，做一些平静的活动，如阅读或听轻柔的音乐，直到感到困倦为止。\n\n如果以上方法不能解决您的问题，最好咨询医生或专业的睡眠医学专家，获取更专业的建议和帮助。')]

## API interface

> **start http service**

```
git clone https://github.com/THUDM/ChatGLM2-6B.git
python api.py

python api.py 
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.56it/s]
INFO:     Started server process [181653]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
[2023-10-18 21:25:50] ", prompt:"你好", response:"'你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。'"
INFO:     127.0.0.1:60868 - "POST / HTTP/1.1" 200 OK



```

> **call ther service**

In [1]:
import requests

In [2]:
def chat(prompt, history):
    response = requests.post(
        url='http://127.0.0.1:8000',
        json={'prompt': prompt, 'history': history},
        headers={'Content-Type': 'application/json;charset=utf-8'}
    )
    return response.json()['response'], response.json()['history']

In [4]:
response, history = chat("你好", history=[])

In [5]:
response

'你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。'

## langchain interface

In [21]:
from langchain.llms import ChatGLM

In [24]:
endpoint_url = 'http://127.0.0.1:8000'
llm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=80000,
    top_p=0.9,
    cache=False)

In [25]:
llm("你好")

'你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。'

## Simple QA

In [1]:
from langchain.document_loaders import DirectoryLoader

In [2]:
from langchain.text_splitter import CharacterTextSplitter

### text splitter

In [3]:
loader = DirectoryLoader('books')

In [4]:
documents = loader.load()

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=0)

In [6]:
split_docs = text_splitter.split_documents(documents=documents)

In [7]:
split_docs[0]

Document(page_content='动物庄园——一个神奇的故事\n\n作者：[英]乔治·奥威尔 著\n\n张毅 高孝先 译\n\n第一章\n\n故事发生在曼纳庄园里。这天晚上，庄园的主人琼斯先生说是已经锁好了鸡棚，但\n\n由于他喝得醉意十足，竟把里面的那些小门都忘了关上。他提着马灯踉踉跄跄地穿过院\n\n子，马灯光也跟着一直不停地晃来晃去，到了后门，他把靴子一脚一只踢了出去，又从\n\n洗碗间的酒桶里舀起最后一杯啤酒，一饮而尽，然后才上床休息。此时，床上的琼斯夫\n\n人已是鼾声如雷了。\n\n等那边庄主院卧室里的灯光一熄灭，整个庄园窝棚里就泛起一阵扑扑腾腾的骚动。', metadata={'source': 'books/animal_fram.txt'})

### Embedding

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [9]:
model_name = '/root/.cache/huggingface/hub/dataroot/models/GanymedeNil/text2vec-large-chinese'

In [10]:
embeddings = HuggingFaceEmbeddings(model_name=model_name, 
                                   model_kwargs={'device': 'cuda:0'},
                                   encode_kwargs={'normalize_embeddings': False})

No sentence-transformers model found with name /root/.cache/huggingface/hub/dataroot/models/GanymedeNil/text2vec-large-chinese. Creating a new one with MEAN pooling.


### VectorStore

In [12]:
from langchain.vectorstores import FAISS

In [13]:
db = FAISS.from_documents(split_docs, embeddings)

In [14]:
query = '故事发生在曼纳庄园里。这天晚上，庄园的主人琼斯先生说是已经锁好了鸡棚'

In [15]:
embedding_vector = embeddings.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)

In [37]:
docs.__len__()

4

In [17]:
print(docs[0].page_content)

动物庄园——一个神奇的故事

作者：[英]乔治·奥威尔 著

张毅 高孝先 译

第一章

故事发生在曼纳庄园里。这天晚上，庄园的主人琼斯先生说是已经锁好了鸡棚，但

由于他喝得醉意十足，竟把里面的那些小门都忘了关上。他提着马灯踉踉跄跄地穿过院

子，马灯光也跟着一直不停地晃来晃去，到了后门，他把靴子一脚一只踢了出去，又从

洗碗间的酒桶里舀起最后一杯啤酒，一饮而尽，然后才上床休息。此时，床上的琼斯夫

人已是鼾声如雷了。

等那边庄主院卧室里的灯光一熄灭，整个庄园窝棚里就泛起一阵扑扑腾腾的骚动。


In [18]:
print(docs[1].page_content)

马嘶嘶地鸣，鸭子嘎嘎地唤。唱着这首歌，他们是多么地兴奋，以至于整整连着唱了五

遍，要不是中途被打断，他们真有可能唱个通宵。

不巧，喧嚣声吵醒了琼斯先生，他自以为是院子中来了狐狸，便跳下床，操起那支

总是放在卧室墙角的猎枪，用装在膛里的六号子弹对着黑暗处开了一枪，弹粒射进大谷

仓的墙里。会议就此匆匆解散。动物们纷纷溜回自己的窝棚。家禽跳上了他们的架子，

家畜卧到了草堆里，顷刻之间，庄园便沉寂下来。

（感谢中译者张毅、高孝先以及本书电子版的输入者复旦大学的唐薇小姐）

第二章


### Retriever

In [19]:
from langchain.chains import RetrievalQA

In [26]:
retriever = db.as_retriever()

In [27]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [34]:
response = qa.run("琼斯先生是谁")

In [35]:
print(response)

琼斯先生是一个人名，是斯诺鲍的密探。


In [36]:
response = qa.run("猪为什么反抗")
print(response)

斯诺鲍和拿破仑认为这些想法与动物主义的精神背道而驰。
